In [ ]:
all_text = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps_qValues_EBA.txt', 'r').readlines()

In [ ]:
q_value_list = []
for line in all_text[1:]:
    q_value_list.append(float(line.split('\t')[-1].strip()))

In [ ]:
from pprint import pprint
print(len(all_text_b))
pprint([a.split('\t') for a in all_text_b])

In [ ]:
all_text_b = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps_qValues_TDA_RandomPeaks.txt', 'r').readlines()


In [ ]:
q_value_list_b = []
for line in all_text_b[1:]:
    q_value_list_b.append(float(line.split('\t')[-1].strip()))

In [ ]:
import pylab as plt
plt.figure(figsize=(8,6))
plt.plot(q_value_list_b, q_value_list, label='random')
plt.plot([0,0.4], [0,0.4], 'k--')
plt.xlabel('True')
plt.ylabel('Estimated')
plt.legend()
plt.grid()

In [ ]:
#calculating the true q_values
con_peaks_all = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-GnpsDecoyConditionalPeaks.txt', 'r').readlines()
cond_peaks_scores = []
for line in con_peaks_all[1:]:
    cond_peaks_scores.append(float(line.split('\t')[-1].strip()))

In [ ]:
print(cond_peaks_scores)

In [ ]:
from pprint import pprint
pprint([a.split('\t') for a in con_peaks_all])

In [ ]:
calculate_q_value(cond_peaks_scores)

In [ ]:
"1S/C13H18ClNO/c1-9(15-13(2,3)4)12(16)10-6-5-7-11(14)8-10/h5-9,15H,1-4H3"=="1S/C13H18ClNO/c1-9(15-13(2,3)4)12(16)10-6-5-7-11(14)8-10/h5-9,15H,1-4H3"

In [ ]:
import sys
sys.path.append(r'C:\Users\Gosia\Desktop\FDR-Metabolomics\src\passatutto_parser.py')
import passatutto_parser as pp
pre_spectrums = pp.PassatuttoParser(r'C:\Users\Gosia\Downloads\MassbankOrbi').parse_folder()

In [ ]:
from matchms.importing.load_from_json import as_spectrum
spectrums = []
for i, s in enumerate( pre_spectrums ):
    spectrums.append(as_spectrum(s))
    if i and i % 10 == 0:
        print('processed %d'%i)

In [ ]:
from pprint import pprint
name_inchi = {}
for s in spectrums:
    name_inchi[s.get("compound_name")] = s.get("inchi")
    
pprint(name_inchi)

In [92]:
file = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps.txt', 'r').readlines()

def cut_inchi(inchi):
    return inchi.split("=")[1].split("-")[0]
    
from collections import namedtuple
Hit = namedtuple('Hit', ['query', 'score', 'false_hit', 'inchi'])

hits = []
for line in file[1:]:
    q,t,ti,s = line.split("\t")
    hit = cut_inchi(name_inchi[q]) != cut_inchi(ti)
    hits.append(Hit(q,s,hit,ti))
    



In [93]:
print(calculate_q_value(hits))

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.012345679012345678, 0.012195121951219513, 0.012048192771084338, 0.011904761904761904, 0.023529411764705882, 0.023255813953488372, 0.022988505747126436, 0.022727272727272728, 0.02247191011235955, 0.022222222222222223, 0.02197802197802198, 0.021739130434782608, 0.021505376344086023, 0.02127659574468085, 0.021052631578947368, 0.020833333333333332, 0.020618556701030927, 0.02040816326530612, 0.030303030303030304, 0.03, 0.0297029702970297, 0.029411764705882353, 0.02912621359223301, 0.028846153846153848, 0.0380952380952381, 0.03773584905660377, 0.037383177570093455, 0.037037037037037035, 0.036697

In [135]:
QVal = namedtuple( 'QVal', [ 'query', 'qval', 'inchi', 'false_hit'])

def calculate_q_value(hits):
    hits.sort(key=lambda x: x.score, reverse=True)
    fdr_vals = []
    for i in range(len(hits)):
        fdr = sum([int(h.false_hit) for h in hits[:i]])/(i+1)
        fdr_vals.append(fdr)
    q_vals = [0 for f in fdr_vals]
    q_vals[-1] = fdr_vals[-1]
    for i in range(len(q_vals)-2,0,-1):
        q_vals[i] = min(fdr_vals[i],q_vals[i+1])
    q_list = []
    for i,h in enumerate(hits):
        #if not h.false_hit: # i.e. the true one
            q_list.append(QVal(h.query, q_vals[i], h.inchi, h.false_hit))
    return fdr_vals, q_vals, q_list

In [136]:
def fdr_q(fdr_table):
    fdr_table.sort(key=lambda x: x[1], reverse=True)
    fdr_vals = []
    queries,s,ss = zip(*fdr_table)
    for i in range(len(fdr_table)):
        fdr = sum(ss[:i])/(i+1)
        fdr_vals.append(fdr)
    q_vals = [0 for f in fdr_vals]
    q_vals[-1] = fdr_vals[-1]
    for i in range(len(q_vals)-2,0,-1):
        q_vals[i] = min(fdr_vals[i],q_vals[i+1])
    q_list = []
    for i,q in enumerate(queries):
        #if ss[i] == 0: # i.e. the true one
            q_list.append((q, q_vals[i]))
    return fdr_vals, q_vals, q_list

In [156]:
random_q_vals = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps_qValues_TDA_RandomPeaks.txt', 'r').readlines()
    
def cut_inchi_2(inchi):
    return inchi.rsplit('/',1)[0]
    
hits = []
name_q_val = {}
for line in random_q_vals[1:]:
    q,t,ti,s,q_val = line.split("\t")
    hit = cut_inchi(name_inchi[q]) != cut_inchi(ti)
    print(q_val.strip(), hit)
    #hit = name_inchi[q] != ti
    name_q_val.setdefault(q, []).append( (q_val,ti) )
    hits.append(Hit(q,s,hit,ti))

0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0038665066910956265 False
0.0038665066910956265 False
0.0038665066910956265 False
0.0038665066910956265 False
0.0038665066910956265 False
0.0038665066910956265 False
0.0038665066910956265 False
0.0038665066910956265 False
0.0038665066910956265 True
0.0038665066910956265 False
0.003866506

In [138]:
from collections import Counter
c = Counter( h.query for h in hits )
pprint( c.most_common(3) )
print(len(hits))

[('Erythromycin', 2), ('Reserpine', 2), ('Adenosine', 2)]
163


In [139]:
import numpy as np
_,_,a = calculate_q_value(hits)
for qval in a:
    print(qval.qval, qval.false_hit)

0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 False
0.0 True
0.011904761904761904 False
0.011904761904761904 False
0.011904761904761904 False
0.011904761904761904 True
0.02040816326530612 False
0.02040816326530612 False
0.02040816326530612 False
0.0204081632653061

In [ ]:
count = 0
for name,q_val,inchi in a:
    if not any( np.isclose(float(q_val),float(n) ) for n,_ in name_q_val[name]):
        
        if not inchi == name_inchi[name]:
            pprint([name,inchi,name_inchi[name],q_val,name_q_val[name]])
        #pprint([name,float(q_val),name_q_val[name]+[(inchi,)]])
        count += 1
print(count,len(a))        

In [157]:
def take_smaller(x):
    if len(x) == 1:
        return float(x[0][0])
    smaller = 0
    for i, (n,_) in enumerate(x):
        if float(n) < float(x[smaller][0]):
            smaller = i
    return float(x.pop(smaller)[0])

count = 0
print('Passatutto,   Ours,      Are They the same?     Name')
for name,q_val,inchi,_ in a:
    print('%4f      %4f    %s            %s' %( take_smaller(name_q_val[name]), float(q_val), cut_inchi(inchi)==cut_inchi(name_inchi[name]),name))
    if not any( np.isclose(float(q_val),float(n) ) for n,_ in name_q_val[name]):
        count += 1
print(count,len(a))        

Passatutto,   Ours,      Are They the same?     Name
0.000000      0.000000    True            Bupropion
0.000000      0.000000    True            Ketoprofen
0.000000      0.000000    True            Sotalol
0.000000      0.000000    True            Flufenamic acid
0.000000      0.000000    True            Moclobemide
0.000000      0.000000    True            Mefenamic acid
0.000000      0.000000    True            Gabapentin
0.000000      0.000000    True            Venlafaxine
0.000000      0.000000    True            Clarithromycin
0.000000      0.000000    True            Metoclopramide
0.000000      0.000000    True            Fluconazole
0.000000      0.000000    True            Terbutryn
0.000000      0.000000    True            Clozapine
0.000000      0.000000    True            1-(3-(Trifluoromethyl)phenyl)piperazine
0.000000      0.000000    True            Sulfathiazole
0.000000      0.000000    True            Amisulpride
0.000000      0.000000    True            Sulfapyrid